In [1]:
#import required packages
import numpy as np
import pandas as pd
import nltk
import tamil
import matplotlib.pyplot as plt

In [2]:
# The first piece of code reads the file containing the enrtire set of Thirukkural from my git repo
pd_kural = pd.read_csv('https://raw.githubusercontent.com/krisrs/kural_analysis_py/master/kural_dat_utf8.txt',index_col=0,sep=',',header='infer')

In [3]:
# creating a list from the pandas dataframe to play around a bit# creati 
kural_list = list(pd_kural["kural"])

In [4]:
# Had earlier created uni/bi/trigrams in R using tm package. Though not the main goal here, 
# just touched a bit using nltk package for python
ng_test = nltk.ngrams(kural_list[0].split(" "),2)
for grams in ng_test : print(grams)

('அகர', 'முதல')
('முதல', 'எழுத்தெல்லாம்')
('எழுத்தெல்லாம்', 'ஆதி')
('ஆதி', 'பகவன்')
('பகவன்', 'முதற்றே')
('முதற்றே', 'உலகு')


In [5]:
# Had earlier created uni/bi/trigrams in R using tm package. Though not the main goal here, 
# just touched a bit using nltk package for python
tri_array = []
for i in range(0,3) : 
    ng = nltk.ngrams(kural_list[i].split(" "),3)
    for grams in ng : 
        tri_array.append(grams)
print(tri_array)

[('அகர', 'முதல', 'எழுத்தெல்லாம்'), ('முதல', 'எழுத்தெல்லாம்', 'ஆதி'), ('எழுத்தெல்லாம்', 'ஆதி', 'பகவன்'), ('ஆதி', 'பகவன்', 'முதற்றே'), ('பகவன்', 'முதற்றே', 'உலகு'), ('கற்றதனால்', 'ஆய', 'பயனென்கொல்'), ('ஆய', 'பயனென்கொல்', 'வாலறிவன்'), ('பயனென்கொல்', 'வாலறிவன்', 'நற்றாள்'), ('வாலறிவன்', 'நற்றாள்', 'தொழாஅர்'), ('நற்றாள்', 'தொழாஅர்', 'எனின்'), ('மலர்மிசை', 'ஏகினான்', 'மாணடி'), ('ஏகினான்', 'மாணடி', 'சேர்ந்தார்'), ('மாணடி', 'சேர்ந்தார்', 'நிலமிசை'), ('சேர்ந்தார்', 'நிலமிசை', 'நீடுவாழ்'), ('நிலமிசை', 'நீடுவாழ்', 'வார்')]


In [6]:
# building a vocabulary of all words in the Thirukkural to get the frequent words
kural_vocab = []
for i in range(len(kural_list)) :
    k = kural_list[i].split()
    kural_vocab.extend(k)

In [7]:
# just tried creating unigram directly in python without using nltk package
wc=[]
for i in range(len(kural_vocab)) :
    c=0
    for j in range(len(kural_list)) :
        c = c + kural_list[j].count(kural_vocab[i])
    wc.append([kural_vocab[i],c])

In [8]:
# count of vocab - its common knowledge for Tamil folk that each of the 1330 couplets in Thirukkural is approx 7 words each.
# Just a sniff test to check that consistency.. almost there, though not exact - ok for the high level analysis
len(wc)

9391

In [9]:
# Had put in the unigram words and counts into a dataframe to weed out some of the stems and split words - 
# In the next few steps, you would see me doing this manually a bit since the stopwords relevant for this is not known as yet
pd_unicount = pd.DataFrame(data=wc)

In [10]:
pd_unicount.head()
pd_unicount.columns = ["word","count"]
pd_unicount = pd_unicount.drop_duplicates()
pd_unicount.head()

,word,count
0,அகர,1
1,முதல,7
2,எழுத்தெல்லாம்,1
3,ஆதி,2
4,பகவன்,1


In [11]:
pd_unicount.describe()


,count
count,6374.000000
mean,3.761374
std,38.482619
min,1.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,1912.000000


In [12]:
pd_unicount.query('count==1912')

,word,count
389,ம்,1912


In [13]:
pd_unicount = pd_unicount[pd_unicount.word != "தி"]
pd_unicount.describe()

,count
count,6374.000000
mean,3.761374
std,38.482619
min,1.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,1912.000000


In [14]:
pd_unicount.query('count==1717')

,word,count
2894,ட,1717


In [15]:
pd_unicount = pd_unicount[pd_unicount.word != "ட"]
pd_unicount.describe()

,count
count,6373.000000
mean,3.492547
std,31.944231
min,1.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,1912.000000


In [16]:
pd_unicount.query('count==1314')

,word,count
5623,ர்,1314


In [17]:
pd_unicount = pd_unicount[pd_unicount.word != "ர்"]
pd_unicount.describe()

,count
count,6372.000000
mean,3.286880
std,27.404049
min,1.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,1912.000000


In [18]:
pd_unicount.query('count==736')

,word,count
1725,கு,736


In [19]:
pd_unisort=pd_unicount.sort_values(by='count',ascending=0)
pd_unisort.head()

,word,count
389,ம்,1912
1725,கு,736
2463,க்கு,331
1039,கண்,233
333,கி,199


In [20]:
# wanted to get a visual feel for a few of the high frequency words 
test = pd_unisort.query('count>10 and count <=500')

In [21]:
test_x=list(test["word"])
test_y=list(test["count"])
test_s=list(test["count"])

In [22]:
#!pip install plotly

In [23]:
import plotly.plotly as py
import plotly.graph_objs as go

trace0 = go.Scatter(
    x= test_x,
    y= test_y,
    mode='markers',
    marker=dict(
        size=test_s,
    )
)
layout = dict(title = "Plot of high frequency words in the ancient Tamil work Thirukkural </br> \
(restricted from 50 to 500 occurrences) </br> That looks like a Tornado!")
data = [trace0]
fig = dict(data = data,layout = layout)
py.iplot(fig,filename='kural_high_freq_words_50_to_500')

Aw, snap! We didn't get a username with your request.

Don't have an account? https://plot.ly/api_signup

Questions? accounts@plot.ly


PlotlyError: Because you didn't supply a 'file_id' in the call, we're assuming you're trying to snag a figure from a url. You supplied the url, '', we expected it to start with 'https://plot.ly'.
Run help on this function for more information.

In [24]:
# back to the core - getting the Tamil letter equivalents of vowels & consontants into a list to count their occurrence 
# Vowel equivalent in Tamil is called 'uyir' - which literally means 'life'
uyir = ['அ','ஆ','இ','ஈ','உ','ஊ','எ','ஏ','ஐ','ஒ','ஓ','ஔ']
len(uyir)

12

In [25]:
# Consonant equivalent in Tamil is called 'mey' - which literally means 'body'
mey = ['க்','ங்','ச்','ஞ்','ட்','ண்','த்','ந்','ப்','ம்','ய்','ர்','ல்','வ்','ழ்','ள்','ற்','ன்']
len(mey)

18

In [26]:
# unlike in English where vowels and consonants make up the full character set for the language, Tamil has 
# uyir-mey a 12 x 18 matrix combination of uyir and mey that lend a rich 'body full of life' character set for the language
# There's also a singleton letter in the Tamil language - we'll leave him alone (pun!) for now
# so, in all uyir (12), mey (18), and uyirmey(216) and the singleton (aydham - 1) form the native Tamil character set. 
# Later day works have a few more due to the influence of sanskrit - none of that is in this core Tamil work. We'll ignore those 
uyirmey = ['க','கா','கி','கீ','கு','கூ','கெ','கே','கை','கொ','கோ','கௌ','ங','ஙா','ஙி','ஙீ','ஙு','ஙூ','ஙெ','ஙே','ஙை','ஙொ','ஙோ','ஙௌ','ச','சா','சி','சீ','சு','சூ','செ','சே','சை','சொ','சோ','சௌ','ஞ','ஞா','ஞி','ஞீ','ஞு','ஞூ','ஞெ','ஞே','ஞை','ஞொ','ஞோ','ஞௌ','ட','டா','டி','டீ','டு','டூ','டெ','டே','டை','டொ','டோ','டௌ','ண','ணா','ணி','ணீ','ணு','ணூ','ணெ','ணே','ணை','ணொ','ணோ','ணௌ','த','தா','தி','தீ','து','தூ','தெ','தே','தை','தொ','தோ','தௌ','ந','நா','நி','நீ','நு','நூ','நெ','நே','நை','நொ','நோ','நௌ','ப','பா','பி','பீ','பு','பூ','பெ','பே','பை','பொ','போ','பௌ','ம','மா','மி','மீ','மு','மூ','மெ','மே','மை','மொ','மோ','மௌ','ய','யா','யி','யீ','யு','யூ','யெ','யே','யை','யொ','யோ','யௌ','ர','ரா','ரி','ரீ','ரு','ரூ','ரெ','ரே','ரை','ரொ','ரோ','ரௌ','ல','லா','லி','லீ','லு','லூ','லெ','லே','லை','லொ','லோ','லௌ','வ','வா','வி','வீ','வு','வூ','வெ','வே','வை','வொ','வோ','வௌ','ழ','ழா','ழி','ழீ','ழு','ழூ','ழெ','ழே','ழை','ழொ','ழோ','ழௌ','ள','ளா','ளி','ளீ','ளு','ளூ','ளெ','ளே','ளை','ளொ','ளோ','ளௌ','ற','றா','றி','றீ','று','றூ','றெ','றே','றை','றொ','றோ','றௌ','ன','னா','னி','னீ','னு','னூ','னெ','னே','னை','னொ','னோ','னௌ']
len(uyirmey)

216

In [27]:
aydham = ['ஃ']

In [28]:
# printed out the first couplet in the list to start counting the letters
kural_list[0]

'அகர முதல எழுத்தெல்லாம் ஆதி பகவன் முதற்றே உலகு'

In [29]:
# counting characters in the first kural - quick visual check shows simple counts uyirmey characters twice,
# once for their uyir form and again for the mey form
len(kural_list[0])

45

In [30]:
# this is worked around by using the open-tamil package which has get_letters function that gets the exact letters# this i 
len(tamil.utf8.get_letters(kural_list[0]))

32

In [31]:
tamil.utf8.get_letters(kural_list[0][1])[0] == mey[0]


False

In [32]:
# counting the frequency of occurrence of uyir letters in all 1330 couplets in Thirukkural. 
# Interesting to note the last vowel 'ஔ' is not used at all
uyir_count = []
for u in range(len(uyir)) :
    count = 0
    for k in range(len(kural_list)) :
        for l in range(len(kural_list[k])) :
            if tamil.utf8.get_letters(kural_list[k][l])[0] == uyir[u] :
                count = count + 1
    uyir_count.append([uyir[u],count])
print(uyir_count)

[['அ', 834], ['ஆ', 204], ['இ', 670], ['ஈ', 49], ['உ', 584], ['ஊ', 71], ['எ', 451], ['ஏ', 65], ['ஐ', 19], ['ஒ', 201], ['ஓ', 32], ['ஔ', 0]]


In [33]:
# counting the frequency of occurrence of mey letters in all 1330 couplets in Thirukkural.
mey_count = []
for m in range(len(mey)):
    all_count = 0
    for k in range(len(kural_list)):
        this_count = kural_list[k].count(mey[m],0,len(kural_list[k])-1)
        all_count = all_count + this_count
    mey_count.append([mey[m],all_count])
print(mey_count)

[['க்', 976], ['ங்', 261], ['ச்', 184], ['ஞ்', 189], ['ட்', 273], ['ண்', 692], ['த்', 708], ['ந்', 595], ['ப்', 588], ['ம்', 1735], ['ய்', 383], ['ர்', 1187], ['ல்', 1308], ['வ்', 49], ['ழ்', 141], ['ள்', 431], ['ற்', 808], ['ன்', 2005]]


In [34]:
# getting the count of the singleton 'ஃ'
ay_count = 0
for k in range(len(kural_list)):
    count = kural_list[k].count(aydham[0],0,len(kural_list[k])-1)
    ay_count = ay_count + count
ay_count

51

In [35]:
# converting the list of uyirmey letters into a 18 x 12 array 
umey = np.reshape(np.array(uyirmey),(18,-1))

In [36]:
umey

array([['க', 'கா', 'கி', 'கீ', 'கு', 'கூ', 'கெ', 'கே', 'கை', 'கொ', 'கோ',
        'கௌ'],
       ['ங', 'ஙா', 'ஙி', 'ஙீ', 'ஙு', 'ஙூ', 'ஙெ', 'ஙே', 'ஙை', 'ஙொ', 'ஙோ',
        'ஙௌ'],
       ['ச', 'சா', 'சி', 'சீ', 'சு', 'சூ', 'செ', 'சே', 'சை', 'சொ', 'சோ',
        'சௌ'],
       ['ஞ', 'ஞா', 'ஞி', 'ஞீ', 'ஞு', 'ஞூ', 'ஞெ', 'ஞே', 'ஞை', 'ஞொ', 'ஞோ',
        'ஞௌ'],
       ['ட', 'டா', 'டி', 'டீ', 'டு', 'டூ', 'டெ', 'டே', 'டை', 'டொ', 'டோ',
        'டௌ'],
       ['ண', 'ணா', 'ணி', 'ணீ', 'ணு', 'ணூ', 'ணெ', 'ணே', 'ணை', 'ணொ', 'ணோ',
        'ணௌ'],
       ['த', 'தா', 'தி', 'தீ', 'து', 'தூ', 'தெ', 'தே', 'தை', 'தொ', 'தோ',
        'தௌ'],
       ['ந', 'நா', 'நி', 'நீ', 'நு', 'நூ', 'நெ', 'நே', 'நை', 'நொ', 'நோ',
        'நௌ'],
       ['ப', 'பா', 'பி', 'பீ', 'பு', 'பூ', 'பெ', 'பே', 'பை', 'பொ', 'போ',
        'பௌ'],
       ['ம', 'மா', 'மி', 'மீ', 'மு', 'மூ', 'மெ', 'மே', 'மை', 'மொ', 'மோ',
        'மௌ'],
       ['ய', 'யா', 'யி', 'யீ', 'யு', 'யூ', 'யெ', 'யே', 'யை', 'யொ', 'யோ',
        'யௌ'],
       ['ர', 'ரா', 'ரி', 'ரீ', '

In [37]:
import io
print (io.DEFAULT_BUFFER_SIZE)

8192
